### 还有子集问题，多维数组初始化问题。



## 1. 0-1背包问题

给你一个可装载重量为 W 的背包和 N 个物品，每个物品有重量和价值两个属性。
其中第 i 个物品的重量为 wt[i]，价值为 val[i]，现在让你用这个背包装物品，
最多能装的价值是多少？

状态转移方程为：

$$F[i, v]=\max \left\{F[i-1, v], F\left[i-1, v-C_{i}\right]+W_{i}\right\}$$

取决于你第i个物品拿不拿。

In [9]:

def knapsack01(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(capacity+1)] for _ in range(n+1)]
    for i in range(1, n+1):
        for j in range(capacity+1):
            dp[i][j] = dp[i-1][j]
            if j >= weights[i-1]:
                dp[i][j] = max(dp[i][j], dp[i-1][j-weights[i-1]] + values[i-1])

    # print
    val = dp[n][capacity]
    res = capacity
    for i in range(n, 0, -1):
        if val != dp[i-1][res]:
            print(weights[i-1], values[i-1])
            val -= values[i-1]
            res -= weights[i-1]
    return dp[n][capacity]

In [10]:
val = knapsack01([4, 10, 5, 6],  [2, 1, 4, 3], 6)
print(val)

3 6
1 10
2 4
20


因为dp[i][j]只用到了dp[i-1]相关的东西，我们可以只保留一行。如果倒着运算，dp[j]
用到的是dp[j-weights[i-1]]，这个还没有被更新，是之前i-1时候算的值。所以



In [11]:
def knapsack01_compress(values, weights, capacity):
    n = len(values)
    # 带上备忘录，知道哪个物品被装了
    dp = [0 for _ in range(capacity+1)]
    memo = [[] for _ in range(capacity+1)]
    for i in range(1, n+1):
        for j in range(capacity, weights[i-1]-1, -1):
            dp[j] = max(dp[j], dp[j-weights[i-1]] + values[i-1])
    return dp[capacity]

In [97]:
val = knapsack01_compress([4, 10, 5, 6],  [2, 1, 4, 3], 7)
print(val)

20


然而，不管是滚动数组还是一维数组，在压缩空间的同时，其带来的缺点也很明显，那就是无法通过
回溯的方式将所有的最优解求出。若要通过回溯方式求出最优解，那必须对所有的历史状态都进行保留。
而压缩存储空间的同时也压缩了问题的解空间，滚动数组的解空间最多仅能容纳2个物品的解，
而一维数组的解空间最多只能容纳1个。



注意，以上题目要求的是不超过背包容量，而不是恰好装满背包。如果要求恰好装满背包，
那初始化就不一样了。

In [12]:
def knapsack01_full_capacity(values, weights, capacity):
    n = len(values)
    MIN = float('-inf')
    dp = [[MIN for _ in range(capacity+1)] for _ in range(n+1)]
    dp[0][0] = 0
    for i in range(1, n+1):
        for j in range(capacity+1):
            dp[i][j] = dp[i-1][j]
            """
            In python, MIN == MIN + anything.
            """
            if j >= weights[i-1]:
                dp[i][j] = max(dp[i][j], dp[i-1][j-weights[i-1]] + values[i-1])
    return dp[n][capacity]

In [15]:
val = knapsack01_full_capacity([4, 10, 5, 6],  [2, 1, 4, 3], 11)
print(val)



-inf


## 1. 完全背包问题


有 $N$ 种物品和一个容量为 $V$ 的背包，每种物品都有无限件可用。放入第 i 种物品 的费用是 $C_i$，
价值是 $W_i$。求解:将哪些物品装入背包，可使这些物品的耗费的费用总 和不超过背包容量，且价值总和最大。

状态转移方程为：

$$F[i, v]=\max \left\{F\left[i-1, v-kC_{i}\right]+kW_{i}\right|0\leq kC_i\leq v\}$$

取决于你第i个物品拿几个。

In [130]:
def knapsack_full(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(capacity+1)] for _ in range(n+1)]
    for i in range(1, n+1):
        for j in range(capacity+1):
            for k in range(0, j//weights[i-1]+1):
                dp[i][j] = max(dp[i][j], dp[i-1][j-k*weights[i-1]] + k*values[i-1])
    return dp[n][capacity]

In [131]:
print(knapsack_full([4, 10, 5, 6],  [2, 1, 4, 3], 8))

80


发现一个问题


\begin{align}
F[i, v-C_i] &=\max \left\{F\left[i-1, (v-C_i) -tC_i\right]+tW_{i}\right|0\leq tC_i\leq v-C_i\}\\
F[i, v] &= \max \left\{F\left[i-1, v-kC_{i}\right]+kW_{i}\right|0\leq kC_i\leq v\}\\
        &=\max \left\{F\left[i-1, v\right], F\left[i-1, v-kC_{i}\right]+kW_{i}\right|C_i\leq kC_i\leq v\}\\
        &=\max \left\{F\left[i-1, v\right], F\left[i-1, v-(t+1)C_{i}\right]+kW_{i}\right|C_i\leq (t+1)C_i\leq v\}\\
      &=\max \left\{F\left[i-1, v\right], F[i, v-C_i] \right\}\\
\end{align}

In [16]:
def knapsack_full_speedup(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(capacity+1)] for _ in range(n+1)]
    for i in range(1, n+1):
        for j in range(capacity+1):
            dp[i][j] = dp[i-1][j]
            if j >= weights[i-1]:
                dp[i][j] = max(dp[i][j], dp[i][j-weights[i-1]] + values[i-1])

    # print

    val = dp[n][capacity]
    res = capacity
    for i in range(n, 0, -1):
        k = 0
        while val != dp[i-1][res]:
            val -= values[i-1]
            res -= weights[i-1]
            k += 1
        print(weights[i-1], k)
    return dp[n][capacity]

In [17]:
print(knapsack_full_speedup([4, 10, 5, 6],  [2, 1, 4, 3], 8))



3 0
4 0
1 8
2 0
80


In [138]:
def knapsack_full_optimize_memo(values, weights, capacity):
    n = len(values)
    dp = [0 for _ in range(capacity+1)]
    for i in range(1, n+1):
        for j in range(weights[i-1], capacity+1):
            dp[j] = max(dp[j], dp[j-weights[i-1]]+values[i-1])
    return dp[capacity]


In [139]:
print(knapsack_full_optimize_memo([4, 10, 5, 6],  [2, 1, 4, 3], 8))

80


### 322. 换硬币
You are given coins of different denominations and a total amount of money amount. Write a function to compute the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

You may assume that you have an infinite number of each kind of coin.

Example 1:

* Input: coins = [1,2,5], amount = 11
* Output: 3
* Explanation: 11 = 5 + 5 + 1

In [1]:
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        MAX = float('inf')
        dp = [MAX for _ in range(amount+1)]
        dp[0] = 0
        for coin in coins:
            for j in range(coin, amount+1):
                dp[j] = min(dp[j], dp[j-coin] + 1)
        return dp[amount] if dp[amount] != MAX else -1

NameError: name 'List' is not defined

### 416. Partition problem

In [1]:
class Solution:

    def canPartition(self, nums: List[int]) -> bool:
        if not nums: return True
        target = sum(nums)
        if target % 2 == 1: return False
        target //= 2
        dp = [False for _ in range(target+1)]
        dp[0] = True
        for n in nums:
            for j in range(target, n-1, -1):
                dp[j] = dp[j] or dp[j-n]
        return dp[target]

NameError: name 'List' is not defined

## 39. Combination Sum


Given an array of distinct integers candidates and a target integer target, return a list of all unique combinations of candidates where the chosen numbers sum to target. You may return the combinations in any order.

The same number may be chosen from candidates an unlimited number of times. Two combinations are unique if the frequency of at least one of the chosen numbers is different.

It is guaranteed that the number of unique combinations that sum up to target is less than 150 combinations for the given input.


In [55]:
class Solution:
    def combinationSum(self, candidates, target: int):
        n = len(candidates)
        dp = [[] for _ in range(target+1)]
        dp[0] = [[]]
        for i in range(1, n+1):
            s = candidates[i-1]
            for j in range(s, target+1):
                dp[j] = dp[j] + [res + [s] for res in dp[j-s]]
        return dp[target]

In [56]:
Solution().combinationSum(candidates = [2,3,6,7], target = 7)

[[2, 2, 3], [7]]